## Output Parsers
Language models output text. But many times you may want to get more structured information than just text back. This is where output parsers come in.
Output parsers are classes that help structure language model responses. There are 2 main methods an output parser must implement:
- **Get format instructions**: A method which returns a string containing instructions for how the output of a language model should be fommatted.
- **Parse**: A method which takes in a string (assumed to be the response from a language model) and parses it into some structure.

In [1]:
from langchain.output_parsers import CommaSeparatedListOutputParser
from langchain.prompts import PromptTemplate
from langchain_google_genai import GoogleGenerativeAI
from dotenv import load_dotenv
import os

e:\hobby_project\gen-ai\demo_langchain\RAG\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()

True

In [3]:
model = GoogleGenerativeAI(
  model="gemini-1.5-pro-latest", 
  google_api_key=os.getenv("GOOGLE_API_KEY"), 
)

### CSV Parser
This output parser can be used when you want to return a list of comma-separated items.

In [4]:
output_parser = CommaSeparatedListOutputParser()

format_instructions = output_parser.get_format_instructions()
prompt = PromptTemplate(
    template="List five places {places}.\n{format_instructions}",
    input_variables=["places"],
    partial_variables={"format_instructions": format_instructions},
)

In [5]:
chain = prompt | model | output_parser

In [6]:
chain.invoke({"places": "for summer tourism in Finland"})

['Helsinki', 'Turku Archipelago', 'Tampere', 'Lake Saimaa', 'Rovaniemi']

### JSON parser
This output parser allows users to specify a JSON schema and query LLMs for outputs that conform to that schema. Keep in mind that large language models are leaky abstractions! You'll have to use an LLM with sufficient capacity to generate well-formed JSON. In the OpenAI family, DaVinci can do reliably but Curie’s ability already drops off dramatically.

#### The following example uses Pydantic to declare your data model.

In [7]:
from typing import List
from langchain_core.output_parsers import JsonOutputParser

In [12]:
# Define your desired data structure.
class Travel(BaseModel):
    place: str = Field(description="name of the places")
    description: str = Field(description="description of the place")
    activities: str = Field(description="what to do in that place")   

In [ ]:
# And a query intented to prompt a language model to populate the data structure.
travel_query = "Suggest a place in India for going on a trip this summer to avoid heat."

# Set up a parser + inject instructions into the prompt template.
parser = JsonOutputParser(pydantic_object=Travel)

prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

chain = prompt | model | parser

chain.invoke({"query": travel_query})